In [1]:
import requests             
from bs4 import BeautifulSoup 
import csv                  
import webbrowser
import io

def display(content, filename='output.html'):
    with open(filename, 'wb') as f:
        f.write(content)
        webbrowser.open(filename)

def get_soup(session, url, show=False):
    r = session.get(url)
    if show:
        display(r.content, 'temp.html')

    if r.status_code != 200: # not OK
        print('[get_soup] status code:', r.status_code)
    else:
        return BeautifulSoup(r.text, 'html.parser')
    
def post_soup(session, url, params, show=False):
    '''Read HTML from server and convert to Soup'''

    r = session.post(url, data=params)
    
    if show:
        display(r.content, 'temp.html')

    if r.status_code != 200: # not OK
        print('[post_soup] status code:', r.status_code)
    else:
        return BeautifulSoup(r.text, 'html.parser')
    
def scrape(url, lang='ALL'):

    # create session to keep all cookies (etc.) between requests
    session = requests.Session()

    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:57.0) Gecko/20100101 Firefox/57.0',
    })


    items = parse(session, url + '?filterLang=' + lang)

    return items

def parse(session, url):
    '''Get number of reviews and start getting subpages with reviews'''

    print('[parse] url:', url)

    soup = get_soup(session, url)

    if not soup:
        print('[parse] no soup:', url)
        return

    num_reviews = soup.find('span', class_='reviews_header_count').text # get text
    num_reviews = num_reviews[1:-1] 
    num_reviews = num_reviews.replace(',', '')
    num_reviews = int(num_reviews) # convert text into integer
    print('[parse] num_reviews ALL:', num_reviews)

    url_template = url.replace('.html', '-or{}.html')
    print('[parse] url_template:', url_template)

    items = []

    offset = 0

    while(True):
        subpage_url = url_template.format(offset)

        subpage_items = parse_reviews(session, subpage_url)
        if not subpage_items:
            break

        items += subpage_items

        if len(subpage_items) < 5:
            break

        offset += 5

    return items

def get_reviews_ids(soup):

    items = soup.find_all('div', attrs={'data-reviewid': True})

    if items:
        reviews_ids = [x.attrs['data-reviewid'] for x in items][::2]
        print('[get_reviews_ids] data-reviewid:', reviews_ids)
        return reviews_ids
    
def get_more(session, reviews_ids):

    url = 'https://www.tripadvisor.com/OverlayWidgetAjax?Mode=EXPANDED_HOTEL_REVIEWS_RESP&metaReferer=Hotel_Review'

    payload = {
        'reviews': ','.join(reviews_ids), # ie. "577882734,577547902,577300887",
        #'contextChoice': 'DETAIL_HR', # ???
        'widgetChoice': 'EXPANDED_HOTEL_REVIEW_HSX', # ???
        'haveJses': 'earlyRequireDefine,amdearly,global_error,long_lived_global,apg-Hotel_Review,apg-Hotel_Review-in,bootstrap,desktop-rooms-guests-dust-en_US,responsive-calendar-templates-dust-en_US,taevents',
        'haveCsses': 'apg-Hotel_Review-in',
        'Action': 'install',
    }

    soup = post_soup(session, url, payload)

    return soup

def parse_reviews(session, url):
    '''Get all reviews from one page'''

    print('[parse_reviews] url:', url)

    soup =  get_soup(session, url)

    if not soup:
        print('[parse_reviews] no soup:', url)
        return

    hotel_name = soup.find('h1', id='HEADING').text

    reviews_ids = get_reviews_ids(soup)
    if not reviews_ids:
        return

    soup = get_more(session, reviews_ids)

    if not soup:
        print('[parse_reviews] no soup:', url)
        return

    items = []

    for idx, review in enumerate(soup.find_all('div', class_='reviewSelector')):

        badgets = review.find_all('span', class_='badgetext')
        if len(badgets) > 0:
            contributions = badgets[0].text
        else:
            contributions = '0'

        if len(badgets) > 1:
            helpful_vote = badgets[1].text
        else:
            helpful_vote = '0'
        user_loc = review.select_one('div.userLoc strong')
        if user_loc:
            user_loc = user_loc.text
        else:
            user_loc = ''
            
        bubble_rating = review.select_one('span.ui_bubble_rating')['class']
        bubble_rating = bubble_rating[1].split('_')[-1]
        bubble_rating = bubble_rating[0]

        item = {
            'Review': review.find('p', class_='partial_entry').text,
            'Review_Date': review.find('span', class_='ratingDate')['title'], # 'ratingDate' instead of 'relativeDate'
            'Review_Rating': bubble_rating,
            }

        items.append(item)
        print('\n--- review ---\n')
        for key,val in item.items():
            print(' ', key, ':', val)

    print()

    return items



In [2]:
def write_in_csv(items, filename='results.csv',
                  headers=['hotel name', 'review title', 'review body',
                           'review date', 'contributions', 'helpful vote',
                           'user name' , 'user location', 'rating'],
                  mode='w'):

    print('--- CSV ---')

    with io.open(filename, mode, encoding="utf-8") as csvfile:
        csv_file = csv.DictWriter(csvfile, headers)

        if mode == 'w':
            csv_file.writeheader()

        csv_file.writerows(items)
        
DB_COLUMN   = 'Review'
DB_COLUMN1 = 'Review_Date'
DB_COLUMN2 = 'Review_Rating'

start_urls = [
    'https://www.tripadvisor.com/Hotel_Review-g60898-d223001-Reviews-Wingate_by_Wyndham_Atlanta_Buckhead-Atlanta_Georgia.html',
]

lang = 'en'

headers = [ 
    DB_COLUMN, 
    DB_COLUMN1, 
    DB_COLUMN2,
]


In [3]:
for url in start_urls:

    # get all reviews for 'url' and 'lang'
    items = scrape(url, lang)

    if not items:
        print('No reviews')
    else:
        # write in CSV
        filename = url.split('Reviews-')[1][:-5] + '__' + lang
        print('filename:', filename)
        write_in_csv(items, filename + '.csv', headers, mode='w')


[parse] url: https://www.tripadvisor.com/Hotel_Review-g60898-d223001-Reviews-Wingate_by_Wyndham_Atlanta_Buckhead-Atlanta_Georgia.html?filterLang=en
[parse] num_reviews ALL: 2475
[parse] url_template: https://www.tripadvisor.com/Hotel_Review-g60898-d223001-Reviews-Wingate_by_Wyndham_Atlanta_Buckhead-Atlanta_Georgia-or{}.html?filterLang=en
[parse_reviews] url: https://www.tripadvisor.com/Hotel_Review-g60898-d223001-Reviews-Wingate_by_Wyndham_Atlanta_Buckhead-Atlanta_Georgia-or0.html?filterLang=en
[get_reviews_ids] data-reviewid: ['635475163', '634055513', '632937390', '631825734', '631495333']

--- review ---

  Review : We have spent a couple of weeks touring round a small part of the United States and have now used Wingate by Wyndham on three occasions and every experience has been excellent. I will definitely use them again on future visits.Brilliant welcome at reception - happy and smiley which is always very welcome and also terrifically helpful.Lovely room. Immaculate in every way 

[get_reviews_ids] data-reviewid: ['623852483', '623177738', '622691449', '622198392', '620819144']

--- review ---

  Review : This is a good, medium priced hotel for an overnight stay. No frills, but clean and quiet. I have stayed a couple of times, no complaints. Breakfast is typical of this type of hotel, but it is adequate. 
  Review_Date : October 10, 2018
  Review_Rating : 4

--- review ---

  Review : I have stayed at the Wingate Buckhead since 2009 and have been amazed at their service, staff and the accommodations always. It is very convenient to GA 400 and you don't have to valet park. The rooms have wood floors with really nice area rugs which just feels cleaner. They offer a Courtesy Car which will take you within 3 miles of the property which is super nice after a long drive in. We could stay anywhere we wish in Buckhead, but we prefer The Wingate. I feel very safe there and look forward to seeing the staff who have become like family over the years. They are a rare proper

[get_reviews_ids] data-reviewid: ['602615591', '601871439', '601620049', '601155805', '601132701']

--- review ---

  Review : My 10 year old son and I had a getaway weekend in Atlanta July 6-9. We had a really terrific weekend and wanted to stay longer, but I had to return to the real world and go back to work. The staff at this hotel are outstanding. Maurice, Monique, Marie, Craig and Ferris are all excellent people and very hospitable to their guests. The breakfast staff was awesome as well as the hot breakfast buffet. My son and I enjoyed ourselves immensely. The staff and hotel provide an environment that is relaxing, easy going, sophisticated and kid friendly. We can’t wait to return to this hotel to experience its greatness.
  Review_Date : August 4, 2018
  Review_Rating : 5

--- review ---

  Review : The great reviews are true! This hotel is a little older and could use a refresh but the excellent staff more than makes up for that. The breakfast is pretty nice too! Beds were v

[get_reviews_ids] data-reviewid: ['593629440', '593557476', '592678484', '591755283', '591749903']

--- review ---

  Review : very nice property, nice breakfast, nice pool, nice hot tub, great location, nice rooms, very good choice for us, free parking, very,very nice property, nice breakfast, nice pool, nice hot tub, great location, nice rooms, very good choice for us, free parking, very, very nice very nice
  Review_Date : July 6, 2018
  Review_Rating : 5

--- review ---

  Review : My girlfriend and myself stayed here again last week while we were in town for the Def Leppard/Journey concert at Suntrust Park. This is a very nice Wingate and a very nice hotel. The staff is extremely courteous and friendly. There are many restaurants and attractions to go explore in the immediate area. Buckhead is an upscale area of Atlanta, so you shouldn't need to worry about your well-being while staying in the area.I recommend this hotel for everyone. Go visit this place and write about your exper

[get_reviews_ids] data-reviewid: ['581055609', '579167413', '578531416', '578511303', '574533733']

--- review ---

  Review : Staff is wonderful. So much professional charm that it makes up for a very noisy room. Just is rather difficult to understand how construction so close to the street does not contain excessive noise proofing plans?
  Review_Date : May 18, 2018
  Review_Rating : 3

--- review ---

  Review : Staying at this location of Wingate by Wyndham was an absolute treat! The internet connection was perfect, breakfast was delicious and everything was comfortable. Everything about my stay was perfect but the most incredible part by far were the people I had the opportunity to be served by. Every person I spoke to was incredibly nice and the front desk staff were especially wonderful. Marquis and Marie served me for the majority of my stay. They went the extra mile every single time they could and went above and beyond to help when I needed it. They were so friendly and helpf

...

[get_reviews_ids] data-reviewid: ['13479127', '13384824', '13259997', '13081126', '12802381']

--- review ---

  Review : Everything really was perfect. The staff, the room, the stretch limo chaffeured transportation, the accomodations (pool/ hot tub/ business area, bar, breakfast buffet...I can go on and on. The staff was amazing. The general manager Anthony made us feel like VIP's during our entire stay. That man knows southern hospitality, to say the least. The cleanliness, elegance and overall look was downright perfect. You can't get a better bang for your buck at this place.
  Review_Date : February 11, 2008
  Review_Rating : 5

--- review ---

  Review : We were at the hotel for four days for a meeting. We were delighted with the pleasantness and wonderful attention from the staff. (The superb breakfast and the really comfortable atmosphere were great, too.) I was most impressed when one of my coworkers found that her father had been airlifted to a trauma center (and later died)

[get_reviews_ids] data-reviewid: ['11478761', '10890071', '10790895', '10736786', '10666286']

--- review ---

  Review : I wanted to write and say thank you to Anthony Hines-Tecoy and his team for making our stay pain free. The hotel is clean and up to date with bar, made to order breakfast, and complimentary area shuttle. The staff is very knowledgable and can recommendreserve good shopping and restaurants nearby. They pretty much are a concierge team, and even set out chocolate chip cookies for all to enjoy.
  Review_Date : December 11, 2007
  Review_Rating : 4

--- review ---

  Review : We stayed here for three nights in October 2007 and can recommend it. What makes it better than the average Inn are:* Breakfast - hot buffet plus omelettes and waffles made to order* Free internet access* Coffee and tea available 24 hours in the lounge* Free limo transportation within three mile radius* Well stocked shop for snacks and other essentials* Room size and fluffiness of bath towels* Frie

[get_reviews_ids] data-reviewid: ['8699755', '8619149', '8458215', '8398917', '8392838']

--- review ---

  Review : Only had two days in Atlanta and picked the Wingate, by far one of the best hotels i have stayed while visiting the US. The staff are fantastic and with Breakfast included plus the free limo service, it really was a great deal. Next time in Atlanta i'm staying here again and would recommend anyone to stay here.
  Review_Date : September 11, 2007
  Review_Rating : 5

--- review ---

  Review : I arrived at Buckhead MARTA station (N7) from the airport, and calledthe hotel to request a free pick up fully expecting to be toldthat as I hadn’t checked in yet I’d have to find my own way there. On the contrary, I was warmly greeted and informed that the driver would be with me momentarily and so she was. Good start. From that moment on, my intitial impressions were only reinforced. The reception staff were friendly and professional, always welcoming and willing to help. The room

[get_reviews_ids] data-reviewid: ['7254610', '7237543', '7154741', '6543873', '6308328']

--- review ---

  Review : Visited this hotel in March of 2007, stayed for a week on a business trip to Atlanta. This was my first stay at a Wingte Inn. What a great hotel - from the start at check in, Anthony at the front desk was very friendly and couteous and checked us in quickly. The rooms are just great - not your typical hotel room as they have hardwood floors so the rooms feel brighter and more open then you typical hotel with dark carpeting with all kinds of odors.I'll let the photos speak for themselves, but the bed was just amazing - a wonderful king bed with amazing linens and covers and lots of pillows. The room had a separte sitting area, desk, and very small kitchenette with microwave and fridge.The bathroom was spacious and best of all the water pressure was very nice and the showerhead was also great, provided lots of water when showeing - not those skimpy shower heads that it tak

[get_reviews_ids] data-reviewid: ['5530363', '4468984', '3976547', '3843705', '1660602']

--- review ---

  Review : I have just returned from my trip to Buckhead. I am from New Jersey and I visit Buckhead once a year to attend Altanta Braves Baseball games. My friends and I have been traveling to Atlanta for the past four years now. We have stayed at the Crowne Plaza and this year we decided to switch to the Wingate Inn. I just want to say that this Hotel is fabulous. We were greeted with a shuttle limo awaiting us at the Marta. The staff is very helpful and the hotel is extremely neat. The rooms, bathroom, pool and bar are absolutely spotless! The beds in the rooms are so comfortable and the sheets and comforter makes you not want to get out of bed. The service to the Marta and anywhere you want to go was on the money. The one thing I have to say is that the breakfast in the morning which includes: homemade waffles and special prepared omelletes not including bacon, sausage muffins e